# Import packages and data 

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy.stats
import anndata
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from scipy import sparse
matplotlib_axes_logger.setLevel('ERROR')

/Users/b8058304/.pyenv/versions/3.6.4/envs/py36/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
sc.settings.verbosity = 1  # verbosity: errors (0), warnings (1), info (2), hints (3)
# Set up the plot config for viewing the annotation clearly.
sc.settings.set_figure_params(dpi=120, dpi_save=1000)
sc.logging.print_versions()

scanpy==1.4.4 anndata==0.7.1 umap==0.3.10 numpy==1.17.1 scipy==1.4.1 pandas==1.0.5 scikit-learn==0.22.2.post1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


# Import the FBM CD4/CD8/Treg

In [3]:
adata = sc.read('/Users/b8058304/Documents/PhD_work/Coding/bm_plus_19pcw/data/bm_plus_19pcw_raw_dr_20201007.h5ad')

In [4]:
adata

AnnData object with n_obs × n_vars = 103228 × 33712 
    obs: 'cell.labels', 'doublets', 'fetal.ids', 'gender', 'is_doublet', 'is_doublet_poptrim', 'is_doublet_wolock', 'lanes', 'nGene', 'nUMI', 'orig.ident', 'percent.mito', 'processing.type', 'scrublet_cluster_score', 'scrublet_score', 'sequencing.type', 'sort.ids', 'april_cell.labels', 'cell.labels_20200708', 'cell.labels_20200713', 'cell.labels_20200718', 'nk_meta', 'mito.threshold'
    var: 'gene_ids-1', 'feature_types-1'
    obsm: 'X_orig_pca', 'X_pca', 'X_umap'

In [5]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CD4 T cell             327
CD8 T cell             171
CD14 monocyte         8763
CD56 bright NK         449
CMP                    425
                      ... 
schwann cells            9
sinusoidal EC          550
stromal macrophage    1464
tDC                    193
tip EC                 362
Length: 64, dtype: int64

In [6]:
from scipy import sparse
array_vals = adata.X
adata.X = sparse.csr_matrix(adata.X)
adata.raw = adata
adata.X = array_vals

In [7]:
adata = adata[adata.obs['cell.labels'].isin(['CD4 T cell', 'CD8 T cell', 'Treg'])].copy()

In [8]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CD4 T cell    327
CD8 T cell    171
Treg           62
dtype: int64

In [9]:
adata.obs["dataset"] = "FBM"

In [10]:
fbm = adata

# Import the thymus DN/DP/CD8/CD4

In [11]:
adata = sc.read('/Users/b8058304/Documents/PhD_work/Coding/thymus/data/fig1.h5ad')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [12]:
adata

AnnData object with n_obs × n_vars = 255901 × 33694 
    obs: 'Anno_level_1', 'Anno_level_2', 'Anno_level_3', 'Anno_level_4', 'Anno_level_5', 'Anno_level_fig1', 'Sample', 'donor', 'organ', 'sort', 'method', 'file', 'Anno_stage', 'Age', 'Gender', 'Source'
    obsm: 'X_umap'

In [13]:
adata.obs

,Anno_level_1,Anno_level_2,Anno_level_3,Anno_level_4,Anno_level_5,Anno_level_fig1,Sample,donor,organ,sort,method,file,Anno_stage,Age,Gender,Source
index,,,,,,,,,,,,,,,,
FCAImmP7179369-AAACCTGAGCCCAATT,T,SP,T_naive,CD8+T,CD8+T,CD8+T,F21_TH_45P,F21,TH,45P,3GEX,FCAImmP7179369,CD8+T_middle,16w,Male,HDBR
FCAImmP7179369-AAACCTGAGCCTATGT,T,SP,T_naive,CD4+T,CD4+T,CD4+T,F21_TH_45P,F21,TH,45P,3GEX,FCAImmP7179369,CD4+T_middle,16w,Male,HDBR
FCAImmP7179369-AAACCTGAGTCGCCGT,T,DP,DP,DP,DP(P),DP,F21_TH_45P,F21,TH,45P,3GEX,FCAImmP7179369,DP_middle,16w,Male,HDBR
FCAImmP7179369-AAACCTGCAGCATGAG,T,SP,T_naive,CD8+T,CD8+T,CD8+T,F21_TH_45P,F21,TH,45P,3GEX,FCAImmP7179369,CD8+T_middle,16w,Male,HDBR
FCAImmP7179369-AAACCTGGTCTCCCTA,T,SP,CD8αα(I),CD8αα(I),CD8αα(I),CD8αα,F21_TH_45P,F21,TH,45P,3GEX,FCAImmP7179369,CD8αα_middle,16w,Male,HDBR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Human_colon_16S7985397-TTTGGTTCAAACCTAC,T,DP,DP,DP,DP(P),DP,F74_TH_TOT_5GEX_2,F74,TH,TOT,5GEX,Human_colon_16S7985397,DP_early,10w,Male,HDBR
Human_colon_16S7985397-TTTGGTTCATGGTAGG,T,DP,DP,DP,DP(Q),DP,F74_TH_TOT_5GEX_2,F74,TH,TOT,5GEX,Human_colon_16S7985397,DP_early,10w,Male,HDBR
Human_colon_16S7985397-TTTGGTTTCAAGGTAA,T,DP,DP,DP,DP(P),DP,F74_TH_TOT_5GEX_2,F74,TH,TOT,5GEX,Human_colon_16S7985397,DP_early,10w,Male,HDBR


In [14]:
cell_numbers = adata.obs.groupby(["Anno_level_fig1"]).apply(len)
cell_numbers

Anno_level_fig1
B_memory       2161
B_naive        2152
B_plasma        479
B_pro/pre       290
CD4+T         14506
CD4+Tmem       2879
CD8+T         13257
CD8+Tmem       1644
CD8αα          6783
DC1            1383
DC2             770
DN            42474
DP            97183
ETP             316
Endo           5753
Epi_GCM2         74
Ery             644
Fb_1           7646
Fb_2           3579
Fb_cycling      524
ILC3            561
Lymph           115
Mac             863
Mast            148
Mgk              36
Mono            265
NK             1615
NKT             349
NMP             185
T(agonist)     3614
TEC(myo)        139
TEC(neuro)      341
Treg           7444
VSMC           3788
aDC             472
cTEC           9411
mTEC(I)        5287
mTEC(II)        808
mTEC(III)       309
mTEC(IV)         44
mcTEC           745
pDC            1048
αβT(entry)    11235
γδT            2582
dtype: int64

In [15]:
adata.obs["cell.labels"] = adata.obs["Anno_level_fig1"]

In [16]:
t_subset = adata[adata.obs['Anno_level_fig1'].isin(['DN', 'DP', 'CD8+T', 'CD4+T', 'Treg'])].copy()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [17]:
cell_numbers = t_subset.obs.groupby(["Anno_level_fig1"]).apply(len)
cell_numbers

Anno_level_fig1
CD4+T    14506
CD8+T    13257
DN       42474
DP       97183
Treg      7444
dtype: int64

In [18]:
t_subset_more = sc.pp.subsample(t_subset, n_obs=4000, random_state=26, copy=True)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [19]:
t_subset_more.shape

(4000, 33694)

In [20]:
cell_numbers = t_subset_more.obs.groupby(["Anno_level_fig1"]).apply(len)
cell_numbers

Anno_level_fig1
CD4+T     345
CD8+T     305
DN        980
DP       2201
Treg      169
dtype: int64

In [21]:
etp_subset = adata[adata.obs['Anno_level_fig1'].isin(['ETP'])].copy()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [22]:
cell_numbers = etp_subset.obs.groupby(["Anno_level_fig1"]).apply(len)
cell_numbers

Anno_level_fig1
ETP    316
dtype: int64

In [23]:
t_subset_more.obs["dataset"] = "thymus"
etp_subset.obs["dataset"] = "thymus"

In [24]:
thymus_dn=t_subset_more
thymus_etp=etp_subset

# Combine the datasets

In [25]:
from scipy import sparse

objects = [thymus_dn, thymus_etp, fbm]

for i in objects:
    array_vals = i.X
    i.X = sparse.csr_matrix(i.X)
    i.raw = i
    i.X = array_vals

In [26]:
merged = thymus_dn.concatenate(fbm, thymus_etp, index_unique=None, join='inner')

Making variable names unique for controlled concatenation.


In [27]:
merged.shape

(4876, 33694)

In [28]:
cell_numbers = merged.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CD4 T cell     327
CD4+T          345
CD8 T cell     171
CD8+T          305
DN             980
DP            2201
ETP            316
Treg           231
dtype: int64

In [29]:
merged.obs["dataset_cell.labels"] = merged.obs["dataset"].astype(str) + '_' + merged.obs["cell.labels"].astype(str)

In [30]:
cell_numbers = merged.obs.groupby(["dataset_cell.labels"]).apply(len)
cell_numbers

dataset_cell.labels
FBM_CD4 T cell     327
FBM_CD8 T cell     171
FBM_Treg            62
thymus_CD4+T       345
thymus_CD8+T       305
thymus_DN          980
thymus_DP         2201
thymus_ETP         316
thymus_Treg        169
dtype: int64

# Pre-process the data

In [31]:
adata=merged

In [32]:
# normalise count data 
sc.pp.normalize_total(adata)

In [33]:
# log the count data 
sc.pp.log1p(adata)

In [34]:
# find variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5) 

In [35]:
# scale the data 
sc.pp.scale(adata, max_value=10)
# scale the negative values in adata.X (which have been normalised, logged and scaled) to zero 
adata.X = np.where(adata.X < 0, 0, adata.X)

# Run some dotplots

In [36]:
subset=adata
subset.obs["dataset_cell.labels"].unique()

array(['thymus_DP', 'thymus_DN', 'thymus_Treg', 'thymus_CD4+T',
       'thymus_CD8+T', 'FBM_CD8 T cell', 'FBM_CD4 T cell', 'FBM_Treg',
       'thymus_ETP'], dtype=object)

In [37]:
subset.obs["dataset_cell.labels"] = subset.obs["dataset_cell.labels"].astype("category")

subset.obs["dataset_cell.labels"] = subset.obs["dataset_cell.labels"].cat.reorder_categories(['thymus_ETP', 
                    'thymus_DN', 'thymus_DP', 'thymus_CD4+T', 'thymus_CD8+T', 'thymus_Treg', 'FBM_CD4 T cell', 
                    'FBM_CD8 T cell', 'FBM_Treg'])

In [38]:
sc.settings.set_figure_params(dpi=1000, dpi_save=1000)

genes = ['CD34', 'IGLL1', 'HIVEP3', 'RGPD3', 'CCND3', 'SMPD3', 'RORC', 'TRAC', 'TRBC2', 'CCR7', 'CD5', 
         'CD4', 'CD40LG', 'CD8A', 'CD8B', 'FOXP3', 'TIGIT']

sc.pl.dotplot(subset, var_names=genes, groupby="dataset_cell.labels", use_raw=False, 
              save="fig3f_tcell_dotplot_20210115.pdf")

... storing 'Age' as categorical
... storing 'Anno_level_1' as categorical
... storing 'Anno_level_2' as categorical
... storing 'Anno_level_3' as categorical
... storing 'Anno_level_4' as categorical
... storing 'Anno_level_5' as categorical
... storing 'Anno_level_fig1' as categorical
... storing 'Anno_stage' as categorical
... storing 'Gender' as categorical
... storing 'Sample' as categorical
... storing 'Source' as categorical
... storing 'april_cell.labels' as categorical
... storing 'cell.labels' as categorical
... storing 'cell.labels_20200708' as categorical
... storing 'cell.labels_20200713' as categorical
... storing 'cell.labels_20200718' as categorical
... storing 'dataset' as categorical
... storing 'donor' as categorical
... storing 'doublets' as categorical
... storing 'fetal.ids' as categorical
... storing 'file' as categorical
... storing 'gender' as categorical
... storing 'is_doublet' as categorical
... storing 'is_doublet_poptrim' as categorical
... storing 'is_dou

GridSpec(2, 5, height_ratios=[0, 10.5], width_ratios=[5.949999999999999, 0, 0.2, 0.5, 0.25])

# Save the data

In [39]:
from scipy.sparse import csr_matrix
adata.X = sparse.csr_matrix(adata.X)

In [40]:
adata.X

<4876x33694 sparse matrix of type '<class 'numpy.float32'>'
	with 10054453 stored elements in Compressed Sparse Row format>

In [41]:
adata.raw.X

<4876x33694 sparse matrix of type '<class 'numpy.float32'>'
	with 10718674 stored elements in Compressed Sparse Row format>

In [42]:
adata.write("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs_mk2/data/fig3f_tcell_fbm_thymus_dotplot_20210115.h5ad")